## Movie Recommender System

In [30]:
import numpy as np 
import pandas as pd

### Importing Datasets


In [31]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [32]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [33]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


### merging the data frames based on title column

In [34]:
movies_merged = movies.merge(credits,on='title')

<p> now we need to create a content based filtering system. For that we need to create tags. Look for columns for creating tags</p>

<pre> List of required columns to create tags:
       1. Genres
       2. id - required while developing website
       3. keywords 
       4. title 
       5. overview - if the summaries of the movies are similar then movies are similar.
       6. cast - some people choose movie based on director or actor
       7. crew
       </pre>

In [35]:
final_movies_frame = movies_merged[['movie_id','title','overview','genres','keywords','cast','crew']]

<pre> The concise dataframe will have 3 colums - movie-id, title, tags.
 tags - overview, genres, keywords, cast and crew will be merged for tags</pre>

### preprocessing the data

In [36]:
final_movies_frame.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [49]:
final_movies_frame.dropna(inplace=True)

c:\Users\SANJANA\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [38]:
final_movies_frame.duplicated().sum()

0

In [39]:
final_movies_frame.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

### Formatting the columns

In [40]:
final_movies_frame.iloc[0].genres
# converting this list of dictionaries to list

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [41]:
## creating a helper function to create list of dictionaries to list of genre tags
import ast
def helper(obj):
    genre_list = []

    for i in ast.literal_eval(obj):
        genre_list.append(i['name'])

    return genre_list



In [42]:
#example
helper('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

# As genre is a string passing it into the function will produce an error as below.

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [20]:
# To sort the above error
# converting the string of list to a list

import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

# we will use this snippet in the above code.

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

<p> Formmating Genres </p>

In [43]:
final_movies_frame['genres'] = final_movies_frame['genres'].apply(helper)

C:\Users\SANJANA\AppData\Local\Temp\ipykernel_27108\4126050108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_movies_frame['genres'] = final_movies_frame['genres'].apply(helper)


In [44]:
## applying the same for keywords

final_movies_frame['keywords']=final_movies_frame['keywords'].apply(helper)

C:\Users\SANJANA\AppData\Local\Temp\ipykernel_27108\16100306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_movies_frame['keywords']=final_movies_frame['keywords'].apply(helper)


In [45]:
final_movies_frame.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [46]:
#keeping the first 3 main actors name -- for tags
def helper2(obj):
    list_cast = []

    counter = 0

    for i in ast.literal_eval(obj):
        if counter != 3:
            list_cast.append(i['name'])
            counter += 1
        else:
            break

    return list_cast

In [47]:
final_movies_frame['cast'] = final_movies_frame['cast'].apply(helper2)

C:\Users\SANJANA\AppData\Local\Temp\ipykernel_27108\2611279967.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_movies_frame['cast'] = final_movies_frame['cast'].apply(helper2)
